In [1]:
import pandas as pd
from gensim import utils
from gensim.models import FastText
from razdel import tokenize
from huggingface_hub import hf_hub_download
import fasttext
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from tqdm import tqdm

In [2]:
data = pd.read_csv("./train_dataset_Датасет/datasets/dataset_1.csv")

for i in range(1, 5):
    new_data = pd.read_csv(f"./train_dataset_Датасет/datasets/dataset_{i}.csv")
    data = pd.concat([data, new_data])

In [4]:
model = FastText.load('fasttext.model')

In [37]:
class MyIter:
    def __iter__(self):
        for index, row in building.iterrows():
            tokens = list(tokenize(row["address1"]))
            yield [_.text for _ in tokens]    

model = FastText(vector_size=128, window=5, min_count=1, workers=30, sg=1, negative=10)
model.build_vocab(corpus_iterable=MyIter())
total_examples = model.corpus_count
model.train(corpus_iterable=MyIter(), total_examples=len(building), epochs=20)

(8, 120)

In [5]:
model = FastText.load('fasttext.model')
vector = np.array(model.wv.vectors_vocab)

In [6]:
keys = model.wv.key_to_index
a = np.array(list(keys.keys()))

In [7]:
a = a.reshape(-1,1)

In [144]:
data = np.concatenate((a, vector), axis = 1)


In [9]:
building = pd.read_csv('./data/train.csv')
building

,address1,address2,label
0,kaka,pisy,1
1,хуй,Жопа,0
2,хуй,уйх,1


In [10]:
b1 = list(building['address1'])
b2 = list(building['address2'])
vec1 = np.zeros((len(b1), 128))
vec2 = np.zeros((len(b2), 128))

In [12]:
for i in range(len(b1)):
    vec1[i] = model.wv.get_vector(b1[i])
    vec2[i] = model.wv.get_vector(b2[i])

In [13]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader

In [14]:
class CustomTextDataset(Dataset):
    def __init__(self, x, y, labels):
        self.labels = labels
        self.x= x
        self.y = y
    def __len__(self):
        return len(self.labels)
    def __getitem__(self, idx):
        label = self.labels[idx]
        address1 = self.x[idx]
        address2 = self.y[idx]
        sample = {"address1": address1, "address2": address2, "label": label}
        return sample

In [15]:
vec1, vec2, labels = torch.tensor(vec1), torch.tensor(vec2), torch.tensor(building['label'])

In [16]:
labels.shape

torch.Size([3])

In [17]:
d = CustomTextDataset(vec1, vec2, building['label'])

In [23]:
for i  in range(len(d)):
    # Print the 'text' data of the batch
    print('x: ', d[i]['address1'].shape)
    # Print the 'class' data of batch
    print('y: ', d[i]['address2'].shape, '\n')
    print('Class data: ', d[i]['label'], '\n')

x:  torch.Size([128])
y:  torch.Size([128]) 

Class data:  1 

x:  torch.Size([128])
y:  torch.Size([128]) 

Class data:  0 

x:  torch.Size([128])
y:  torch.Size([128]) 

Class data:  1 



In [107]:
df.to_pickle("my_data.bin")

In [4]:
model =  FastText(vector_size=128, window=5, min_count=1, workers=30, sg=1, negative=10)

In [6]:
model = FastText.load('fasttext.model')

In [7]:
vector_index = []

for index, row in tqdm(building.iterrows()):
    tokens = list(tokenize(row["short_address"]))
    tokens = [_.text for _ in tokens]
    predict = np.array([model.wv[token] for token in tokens])
    predict = np.mean(predict, axis=0)
    predict = predict / np.linalg.norm(predict)
    vector_index.append((row['id'], predict))

0it [00:00, ?it/s]

166645it [00:47, 3496.10it/s]


In [8]:
def get_id(text):
    vector = model.wv[text] 
    tokens = list(tokenize(text))
    tokens = [_.text for _ in tokens]
    vector = np.array([model.wv[token] for token in tokens])
    vector = np.mean(vector, axis=0)
    vector = vector / np.linalg.norm(vector).reshape(1, -1) 
    max_sym = -1
    best_id = 0
    step = 10000

    for ind in range(0, len(vector_index), step):
        vectors = np.array([i[1] for i in vector_index[ind: ind + step]]).T
        syms = vector @ vectors
        best_local_ind = np.argmax(syms)
        sym = syms[0][best_local_ind]
        if sym > max_sym:
            max_sym = sym
            best_id = vector_index[ind + best_local_ind][0]
    return best_id, max_sym

In [9]:
cnt = 0
ind = 0
for index, row in tqdm(data.iterrows()):
    ind += 1
    best_id, max_sym = get_id(row["address"])
    #print(best_id, row['target_building_id'])
    if best_id == row['target_building_id']:
        cnt += 1
    if ind == 1000:
        break
print(cnt / 1000)

999it [02:18,  7.21it/s]

0.396


In [184]:
print(cnt / 1000)

0.236


In [191]:
model.save("fasttext.model")

In [190]:
!pwd

/work/hack
